### 中醫醫案症狀詞抽取

- model: CRF
- 資料集: 舌象
- 類別標註: OS
- 詞位標注: 無
- 拆句: 沒拆分
- 斷詞: 沒有做
- 詞性標註: 使用中科院的
- 引用辭典: 沒有
- 標點: 沒去

In [1]:
import json
import nltk
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i >= ne_idx and i <= (ne_idx + ne_len - 1):
            return "TON"
    
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
s = ""
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    
    #原文
    case_content = cols[2]
    case_content = case_content.replace(" ","")
    case_content = case_content.replace('#',',')
    sentences = case_content.split("。")
    
    #摘要
    summary_str = cols[3].strip()
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "tongue" in keys:
        main = summary_json["tongue"]
        main_list = getTagListWithString(main)
        
        
        s = s + main + "，"
    
    
    t += len(main_list)
        
    ne_index_list = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list.append((ne_idx,len(ne)))
    
    '''
    if len(main_list) != len(ne_index_list):
        c += 1
        print(case_content)
        print(main_list)
        for i,l in ne_index_list:
            print(case_content[i])
        print("------------")
    '''
    
    texts = []
    for i,w in enumerate(case_content):
        label = getLabelWithWordIndex(i,ne_index_list)
        texts.append((w,label))
        
    docs.append(texts)

print(t)
print(c)
#print(s)

3235
0


In [5]:
s = ""
for x in main_list:
    s = s + x
print(s)

舌苔黄腻


In [6]:
for w in docs[2]:
    print(w[0],w[1])

袁 O
某 O
， O
女 O
， O
4 O
0 O
岁 O
。 O
自 O
发 O
性 O
气 O
胸 O
住 O
院 O
， O
患 O
者 O
多 O
次 O
肺 O
大 O
疱 O
破 O
裂 O
出 O
现 O
气 O
胸 O
， O
本 O
次 O
已 O
入 O
院 O
数 O
日 O
， O
经 O
置 O
管 O
减 O
压 O
， O
仍 O
气 O
促 O
难 O
卧 O
， O
自 O
汗 O
胸 O
闷 O
不 O
己 O
， O
邀 O
我 O
会 O
诊 O
。 O
脉 O
细 O
， O
苔 TON
薄 TON
黄 TON
。 O
处 O
以 O
桂 O
枝 O
加 O
厚 O
朴 O
杏 O
子 O
汤 O
， O
加 O
瓜 O
蒌 O
仁 O
、 O
薤 O
白 O
、 O
儿 O
茶 O
， O
2 O
剂 O
症 O
大 O
减 O
， O
二 O
诊 O
服 O
药 O
6 O
剂 O
而 O
愈 O
。 O


In [7]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 2701
print(ave)

606.4124398370974


In [8]:
print(len(docs))

2107


In [9]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器

data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    
    tagged = []
    for tk in tokens:
        tg = pynlpir.segment(tk)
        tagged.append(tg[0])
    
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gm

In [10]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    
    if postag is None:
        postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [11]:
# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 11903
Seconds required: 1.418

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 67605.224896
Feature norm: 1.000000
Error norm: 109726.521523
Active features: 11899
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.454

***** Iteration #2 *****
Loss: 62184.871251
Feature norm: 1.050947
Error norm: 86346.192268
Active features: 11819
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #3 *****
Loss: 50356.331517
Feature norm: 1.237796
Error norm: 32016.242177
Active features: 10221
Line search trials: 1
Line search step: 1.000000
Seconds required fo

***** Iteration #39 *****
Loss: 5550.379407
Feature norm: 16.927066
Error norm: 3969.669655
Active features: 3609
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.231

***** Iteration #40 *****
Loss: 5495.815858
Feature norm: 17.708019
Error norm: 3093.191576
Active features: 3516
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.231

***** Iteration #41 *****
Loss: 5479.397952
Feature norm: 18.539018
Error norm: 4415.856125
Active features: 3445
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.248

***** Iteration #42 *****
Loss: 5425.259584
Feature norm: 19.027627
Error norm: 822.988012
Active features: 3383
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.232

***** Iteration #43 *****
Loss: 5403.272609
Feature norm: 19.476707
Error norm: 1638.808686
Active features: 3274
Line search trials: 1
Line search step: 1.000000
Seconds re

***** Iteration #84 *****
Loss: 4614.482022
Feature norm: 38.755947
Error norm: 709.385669
Active features: 2281
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.235

***** Iteration #85 *****
Loss: 4610.273520
Feature norm: 38.794598
Error norm: 1227.010176
Active features: 2273
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.238

***** Iteration #86 *****
Loss: 4604.561224
Feature norm: 38.909183
Error norm: 534.304449
Active features: 2286
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.233

***** Iteration #87 *****
Loss: 4601.162041
Feature norm: 38.945559
Error norm: 1006.588946
Active features: 2269
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.236

***** Iteration #88 *****
Loss: 4598.039239
Feature norm: 39.091138
Error norm: 891.105845
Active features: 2261
Line search trials: 1
Line search step: 1.000000
Seconds requ

***** Iteration #126 *****
Loss: 4528.627983
Feature norm: 41.796189
Error norm: 395.539547
Active features: 2100
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.235

***** Iteration #127 *****
Loss: 4527.614647
Feature norm: 41.868302
Error norm: 684.400355
Active features: 2099
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.233

***** Iteration #128 *****
Loss: 4526.327188
Feature norm: 41.914149
Error norm: 248.668268
Active features: 2098
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.231

***** Iteration #129 *****
Loss: 4525.312722
Feature norm: 41.961453
Error norm: 582.446369
Active features: 2096
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #130 *****
Loss: 4523.364505
Feature norm: 41.994819
Error norm: 240.431257
Active features: 2094
Line search trials: 1
Line search step: 1.000000
Seconds r

***** Iteration #169 *****
Loss: 4475.813505
Feature norm: 42.800710
Error norm: 649.862185
Active features: 1969
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #170 *****
Loss: 4474.782231
Feature norm: 42.766078
Error norm: 547.378665
Active features: 1975
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.231

***** Iteration #171 *****
Loss: 4474.007055
Feature norm: 42.729763
Error norm: 584.374101
Active features: 1971
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.232

***** Iteration #172 *****
Loss: 4473.230896
Feature norm: 42.692247
Error norm: 603.816247
Active features: 1973
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.234

***** Iteration #173 *****
Loss: 4472.418218
Feature norm: 42.645110
Error norm: 599.896088
Active features: 1965
Line search trials: 1
Line search step: 1.000000
Seconds r

In [13]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [16]:
# Create a mapping of labels to indices
labels = {"O": 0,"TON": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print("F1 score for training set:\n")
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "TON"]))

print("\n")

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print("F1 score for test set:\n")
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "TON"]))

F1 score for training set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00   1282818
        TON       0.61      0.32      0.42      8881

avg / total       0.99      0.99      0.99   1291699



F1 score for test set:

             precision    recall  f1-score   support

          O       1.00      1.00      1.00    344034
        TON       0.48      0.26      0.33      2187

avg / total       0.99      0.99      0.99    346221



In [17]:
f1_score(truths_test, predictions_test, pos_label=1)

0.3339317773788151

In [31]:
for i,doc in enumerate(X_test):
    print(i,'---------')
    for j,word in enumerate(doc):
        w = word[1].split("=")[1]
        y1 = y_pred_test[i][j]
        y2 = y_test[i][j]
        if 'TON' in y1:
            print(w,y1,y2)
    

#print(X_test[12][4][1].split("=")[1])
#print(y_pred[12][4])
#print(y_test[12][4])

0 ---------
苔 TON TON
白 TON TON
腻 TON TON
1 ---------
苔 TON O
白 TON O
苔 TON O
白 TON O
2 ---------
边 TON TON
有 TON TON
齿 TON TON
印 TON TON
3 ---------
4 ---------
苔 TON O
白 TON O
腻 TON O
苔 TON O
白 TON O
腻 TON O
苔 TON O
白 TON O
5 ---------
舌 TON O
尖 TON O
红 TON O
舌 TON O
苔 TON O
白 TON O
6 ---------
舌 TON TON
苔 TON TON
淡 TON TON
黄 TON TON
薄 TON TON
腻 TON TON
边 TON TON
有 TON TON
齿 TON TON
印 TON TON
7 ---------
8 ---------
9 ---------
舌 TON TON
淡 TON TON
黯 TON TON
苔 TON TON
白 TON TON
滑 TON TON
苔 TON O
白 TON O
滑 TON O
10 ---------
苔 TON TON
薄 TON TON
白 TON TON
而 TON TON
干 TON TON
苔 TON O
薄 TON O
白 TON O
而 TON O
干 TON O
舌 TON O
尖 TON O
红 TON O
苔 TON O
薄 TON O
白 TON O
而 TON O
干 TON O
11 ---------
舌 TON TON
红 TON TON
无 TON TON
苔 TON TON
而 TON TON
干 TON TON
12 ---------
13 ---------
14 ---------
15 ---------
16 ---------
舌 TON TON
苔 TON TON
焦 TON TON
黄 TON TON
17 ---------
18 ---------
19 ---------
20 ---------
21 ---------
苔 TON TON
黄 TON TON
厚 TON O
有 TON O
裂 TON O
纹 TON O
22 ---------
23 ----

232 ---------
233 ---------
234 ---------
舌 TON O
淡 TON O
胖 TON O
嫩 TON O
苔 TON O
黄 TON O
腻 TON O
舌 TON O
淡 TON O
苔 TON O
白 TON O
235 ---------
236 ---------
237 ---------
238 ---------
239 ---------
240 ---------
241 ---------
242 ---------
苔 TON TON
白 TON TON
243 ---------
244 ---------
苔 TON O
净 TON O
苔 TON TON
白 TON TON
滑 TON TON
245 ---------
246 ---------
247 ---------
248 ---------
249 ---------
250 ---------
舌 TON TON
淡 TON TON
苔 TON TON
白 TON TON
251 ---------
252 ---------
舌 TON TON
苔 TON TON
黄 TON TON
腻 TON TON
舌 TON O
净 TON O
253 ---------
舌 TON TON
黯 TON TON
苔 TON TON
白 TON TON
254 ---------
舌 TON TON
淡 TON TON
苔 TON TON
白 TON TON
腻 TON TON
255 ---------
256 ---------
舌 TON TON
淡 TON TON
红 TON TON
苔 TON TON
薄 TON TON
黄 TON TON
257 ---------
舌 TON TON
苔 TON TON
白 TON TON
少 TON TON
津 TON TON
258 ---------
259 ---------
舌 TON O
苔 TON O
白 TON O
舌 TON O
淡 TON O
苔 TON O
白 TON O
260 ---------
苔 TON TON
白 TON TON
腻 TON TON
261 ---------
262 ---------
苔 TON TON
白 TON TON
微 TON TON


In [32]:
k = 0
s = []
for w in X_test[k]: 
    s.append(w[1].split("=")[1])
print(''.join(s))

print("-----")

f = False
for i,word in enumerate(X_test[k]):
    w = word[1].split("=")[1]
    y1 = y_pred_test[k][i]
    y2 = y_test[k][i]
    
    if y1 != y2:
       f = True
        
    print(w,y1,y2)
    
    

金某，女，4岁3个月。初诊：2010年3月23日。主诉：发热伴呕吐、腹痛1日。患儿于1日前饱食后当风，继而出现发热，面赤唇红。就诊当日，患儿午饭后呕吐1次，并伴腹痛隐隐，喜温喜按。现症：发热，恶风无汗，轻咳，咽痛，时有腹痛，恶心，纳呆，大便1日未行，舌质红，苔白腻，脉数。西医诊断：急性上呼吸道感染；中医诊断：感冒，证属风热夹滞。治以清热解表，和胃导滞之法，方拟消食退热汤加减。柴胡、荆芥穗、牛蒡子、豆豉、青蒿、黄芩、知母、槟榔、厚朴、牡丹皮各12g,大黄（后下）6g,甘草6g。2剂，水煎服，日1剂。二诊（2010年3月26日）：患儿药后1日热退，咳嗽较前加重，有痰，夜间晨起为著，胃脘胀闷不舒，叩之如鼓，纳呆，大便干，舌质红，苔薄黄腻，脉数。前方去柴胡、豆豉、牛蒡子，加胡连、莱菔子各10g,瓜蒌15g,芦根20g。3剂，水煎服，日1剂。三诊（2010年3月30日）：患儿服药后，咳止，纳增，遂停药。
-----
金 O O
某 O O
， O O
女 O O
， O O
4 O O
岁 O O
3 O O
个 O O
月 O O
。 O O
初 O O
诊 O O
： O O
2 O O
0 O O
1 O O
0 O O
年 O O
3 O O
月 O O
2 O O
3 O O
日 O O
。 O O
主 O O
诉 O O
： O O
发 O O
热 O O
伴 O O
呕 O O
吐 O O
、 O O
腹 O O
痛 O O
1 O O
日 O O
。 O O
患 O O
儿 O O
于 O O
1 O O
日 O O
前 O O
饱 O O
食 O O
后 O O
当 O O
风 O O
， O O
继 O O
而 O O
出 O O
现 O O
发 O O
热 O O
， O O
面 O O
赤 O O
唇 O O
红 O O
。 O O
就 O O
诊 O O
当 O O
日 O O
， O O
患 O O
儿 O O
午 O O
饭 O O
后 O O
呕 O O
吐 O O
1 O O
次 O O
， O O
并 O O
伴 O O
腹 O O
痛 O O
隐 O O
隐 O O
， O O
喜 O O
温 O O
喜 O O
按 O O
。 O O
现 O O
症 O O
： O O
发 O O
热 O O
， O O
恶 O O
风 O O
无 O